<a href="https://colab.research.google.com/github/mahb97/Wakeifier/blob/main/Wakefyer_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Nothing about this is serious so here comes everybody



# Wakefyer - LoRA fine-tune with conservative text handling

by default, doesn't normalise or strip diacritics (includes ultra-light cleaning toggle)


In [5]:
!pip -q install "transformers==4.44.2" "peft==0.12.0" "accelerate==0.34.2" "datasets==2.20.0" "sentencepiece==0.2.0"
!pip install -U "transformers>=4.46.0"
import os, torch
os.environ["PYTORCH_CUDA_ALLOC_CONF"]="expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True
# this will moan about compatibility...
# Let it train, let it dream, the riverrun needs to backprop through all epochs.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 21.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.4/324.4 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 41.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2024.5.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [1]:
import transformers; print(transformers.__version__)


4.57.1


**A working cell for broke girls with no mula for colab and who dont wanna chug money into the firey pit that is hell that is google that is oppression.**

The Wakefyer fine-tuning architecture constitutes a hybridized LoRA-based low-rank adaptation pipeline for stylistic transformation within a bfloat16/fp16 mixed-precision environment. Its design paradigm is deliberately antithetical to large-scale quantized optimization frameworks such as bitsandbytes, privileging instead interpretability, traceability, and computational minimalism. Conceptually, it operates as a micro-epistemic engine: a system whose purpose is not semantic fidelity but stylistic distortion, functioning as a textual “entropy amplifier” with respect to Finnegans Wake’s logorrheic poetics.

Text ingestion follows a non-normalizing, non-canonical pipeline, thereby maintaining orthographic heterogeneity as an essential stylistic feature. The raw Joycean corpus is tokenized through a fast SentencePiece model and segmented using overlapping sliding windows defined by max_len and stride. This overlap emulates a continuum of narrativity (each window partially repeating the previous one) producing a computational analogue of Joyce’s recursive syntactic rhythm. No linguistic cleaning or lemmatization occurs; lexical noise is treated as signal. The preprocessing stage therefore performs not sanitization but syntactic fractalization, generating a dataset of semi-redundant micro-contexts to condition the model’s representational drift.

In [9]:
# because code should also stream-of-consciousness. avoid the binary bore.

import os, gc, random, sys, glob, re
from pathlib import Path
import torch

# tinyllamas on the mound
BASE_MODEL  = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
RUN_ROOT    = "/content/drive/MyDrive/wakefyer_runs"        # Shards of my heart
OUT_DIR     = f"{RUN_ROOT}/wakefyer_out_nobnb"
TRAIN_TEXT  = f"{RUN_ROOT}/FinnegansWake.txt"
Path(OUT_DIR).mkdir(parents=True, exist_ok=True)

# sweet nothing
max_len, stride         = 256, 128
batch_size, grad_accum  = 1, 32
epochs                  = 3
lr, log_steps           = 1e-4, 10
eval_ratio, seed        = 0.02, 42

# because T4s have fragile little snowflake egos
for n in ["trainer","model","base"]:
    if n in globals(): del globals()[n]
gc.collect(); torch.cuda.empty_cache()
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True,max_split_size_mb:64"
torch.backends.cuda.matmul.allow_tf32 = True
random.seed(seed); torch.manual_seed(seed)
if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed) # summons the devil (no batsandbites; just vibessss coz im a witch)

print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available(): print("GPU:", torch.cuda.get_device_name(0))

thunderwords = [
    "bababadalgharaghtakamminarronnkonnbronntonnerronntuonnthunntrovarrhounawnskawntoohoohoordenenthurnuk",
    "Perkodhuskurunbarggruauyagokgorlayorgromgremmitghundhurthrumathunaradidillifaititillibumullunukkunun",
    "klikkaklakkaklaskaklopatzklatschabattacreppycrottygraddaghsemmihsammihnouithappluddyappladdypkonpkot",
    "Bladyughfoulmoecklenburgwhurawhorascortastrumpapornanennykocksapastippatappatupperstrippuckputtanach",
    "Thingcrooklyexineverypasturesixdixlikencehimaroundhersthemaggerbykinkinkankanwithdownmindlookingated",
    "Lukkedoerendunandurraskewdylooshoofermoyportertooryzooysphalnabortansporthaokansakroidverjkapakkapuk",
    "Bothallchoractorschumminaroundgansumuminarumdrumstrumtruminahumptadumpwaultopoofoolooderamaunsturnup",
    "Pappappapparrassannuaragheallachnatullaghmonganmacmacmacwhackfalltherdebblenonthedubblandaddydoodled",
    "husstenhasstencaffincoffintussemtossemdamandamnacosaghcusaghhobixhatouxpeswchbechoscashlcarcarcaract",
    "Ullhodturdenweirmudgaardgringnirurdrmolnirfenrirlukkilokkibaugimandodrrerinsurtkrinmgernrackinarockar",
]

# hackers of abstraction
from transformers import AutoTokenizer
tok_dir = Path(OUT_DIR, "tokenizer")
if tok_dir.exists():
    tok = AutoTokenizer.from_pretrained(str(tok_dir), use_fast=True, local_files_only=True)
else:
    tok = AutoTokenizer.from_pretrained(BASE_MODEL, use_fast=True)
    if tok.pad_token is None: tok.pad_token = tok.eos_token
    tok.add_tokens(thunderwords, special_tokens=False)
    tok_dir.mkdir(parents=True, exist_ok=True)
    tok.save_pretrained(str(tok_dir))
tok.padding_side = "right"
print("Tokenizer vocab:", len(tok))

# Gospel
if not Path(TRAIN_TEXT).exists():
    print(f"[Wakefyer sobbing] The gospel '{TRAIN_TEXT}' is missing.", file=sys.stderr)
    raise SystemExit(2)
raw_txt = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
if len(raw_txt.strip()) < 1000:
    print("[Wakefyer appaled] Joyce did not write haikus.", file=sys.stderr)
bf16_ok = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8

# (optional) norm thndwds
punct = r"[!?.,:;·—–\-…]*"
def normalize_thunderwords(text: str, words: list[str]) -> str:
    text = re.sub(r"-\s*\n\s*", "", text)
    text = re.sub(r"\n+", " ", text)
    for w in words:
        pattern = re.compile(rf"(?<!\w){re.escape(w)}{punct}")
        text = pattern.sub(w, text)
    return re.sub(r"\s{2,}", " ", text).strip()
raw_txt = normalize_thunderwords(raw_txt, thunderwords)

# in came christ the tiger
from datasets import Dataset
ids = tok(raw_txt, add_special_tokens=False).input_ids
def sliding_windows(txt_ids, block_size=max_len, step=stride):
    items=[]
    for s in range(0, max(1, len(txt_ids)-block_size), step):
        ch = txt_ids[s:s+block_size]
        if ch: items.append({"input_ids": ch, "attention_mask":[1]*len(ch)})
    if not items:
        ch = txt_ids[:block_size]
        items=[{"input_ids":ch, "attention_mask":[1]*len(ch)}]
    return Dataset.from_list(items)
ds = sliding_windows(ids, max_len, stride)
test_n = max(1, int(len(ds)*eval_ratio)) if len(ds) > 1 else 1
splits = ds.train_test_split(test_size=test_n, seed=seed) if len(ds)>1 else {"train":ds, "test":ds}
train_ds, eval_ds = splits["train"], splits["test"]
print("Windows:", len(ds), "| Train:", len(train_ds), "Eval:", len(eval_ds))

# call my daughter LoRA
from transformers import AutoConfig, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

cfg = AutoConfig.from_pretrained(BASE_MODEL)
cfg.pad_token_id = tok.eos_token_id

offload_dir = "/content/offload"; os.makedirs(offload_dir, exist_ok=True)
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    config=cfg,
    dtype=torch.bfloat16 if bf16_ok else torch.float16,  # T4 => fp16
    low_cpu_mem_usage=True,
    device_map="auto",
    offload_folder=offload_dir,
)
# self intergration
if base.get_input_embeddings().num_embeddings != len(tok):
    try:
        base.resize_token_embeddings(len(tok), mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(len(tok))
    base.config.vocab_size = len(tok)

base.config.use_cache = False
if hasattr(base, "gradient_checkpointing_enable"):
    base.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
base.config._attn_implementation = "eager"

lora = LoraConfig(
    r=16, lora_alpha=32, lora_dropout=0.05,
    target_modules=["q_proj","k_proj","v_proj","o_proj","gate_proj","up_proj","down_proj"],
    bias="none", task_type="CAUSAL_LM",
)
model = get_peft_model(base, lora)

# spicyyy
emb = model.get_input_embeddings()
emb.weight.requires_grad = True
if hasattr(model, "lm_head") and hasattr(model.lm_head, "weight"):
    model.lm_head.weight.requires_grad = True

model.print_trainable_parameters()

# I curate
from transformers import DataCollatorForLanguageModeling
collator = DataCollatorForLanguageModeling(tok, mlm=False)

# Compatibility shims
import accelerate
from accelerate import Accelerator
if not getattr(Accelerator, "_wake_patch_unwrap", False):
    Accelerator._wake_orig_unwrap = Accelerator.unwrap_model
    def _wake_unwrap(self, model, *args, **kwargs):
        kwargs.pop("keep_torch_compile", None)  # (ignore unknown kw on older accelerate)
        return Accelerator._wake_orig_unwrap(self, model, *args, **kwargs)
    Accelerator.unwrap_model = _wake_unwrap
    Accelerator._wake_patch_unwrap = True
    print("Patched accelerate.Accelerator.unwrap_model (idempotent).")

import transformers.trainer_pt_utils as pt_utils
if not getattr(pt_utils, "_wake_disable_scaler", False):
    pt_utils.get_grad_scaler = lambda *a, **k: None  # (prevent fp16 unscale crash)
    pt_utils._wake_disable_scaler = True
    print("Disabled Trainer GradScaler hook (idempotent).")

# figure skating is more difficults. Padadakis and Cizeron  World Championships FD 2016.
from transformers.training_args import TrainingArguments as HFTrainingArguments
from transformers import Trainer, TrainingArguments  # you're always the first on the ice
TrainingArguments = HFTrainingArguments  # safety in my home

args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=grad_accum,

    learning_rate=lr,
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    weight_decay=0.01,

    logging_strategy="steps",
    logging_steps=log_steps,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    fp16=False,
    bf16 = False,
    save_safetensors=True,
    report_to=[],
)

# kill mixed precision everywhere
import os
os.environ["ACCELERATE_MIXED_PRECISION"] = "no"   # force Accelerate to 'no' MP

# AVOID MESSY GradScaler
import transformers.trainer_pt_utils as pt_utils
pt_utils.get_grad_scaler = lambda *a, **k: None
from accelerate import Accelerator
Accelerator.unscale_gradients = lambda self, optimizer=None: None


# calls for some Berlin deep house mix
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,   # (keep even if no eval mid-run)
    data_collator=collator,
    tokenizer=tok,
)
trainer.train()

# Save adapter + tokenizer to Drive
adapter_dir = Path(OUT_DIR) / "adapter"
token_dir   = Path(OUT_DIR) / "tokenizer"
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved to Drive:", adapter_dir, token_dir)

# Tinyyy sample (for the vibes) don't be so harsh it's still learning...
prompt = "riverrun, past Eve and Adam's,"
inputs = tok(prompt, return_tensors="pt").to(model.device)
model.eval()
with torch.no_grad():
    out = model.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=1.05,
        top_p=0.92,
        repetition_penalty=1.05,
        pad_token_id=tok.eos_token_id,
    )
print("\n=== SAMPLE ===\n", tok.decode(out[0], skip_special_tokens=True))


Torch: 2.8.0+cu126
CUDA available: True
GPU: Tesla T4
Tokenizer vocab: 32010


Token indices sequence length is longer than the specified maximum sequence length for this model (396323 > 2048). Running this sequence through the model will result in indexing errors


Windows: 3095 | Train: 3034 Eval: 61


/tmp/ipython-input-178213077.py:208: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The model is already on multiple devices. Skipping the move to device specified in `args`.
The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


trainable params: 143,728,640 || all params: 1,112,705,024 || trainable%: 12.9170


Step,Training Loss
10,694.765000
20,502.750600
30,244.705900
40,111.735400
50,181.131900
60,164.869700
70,140.357800
80,146.524600
90,159.336900
100,180.360400


/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/utils/save_and_load.py:232: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetunin

Saved to Drive: /content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb/adapter /content/drive/MyDrive/wakefyer_runs/wakefyer_out_nobnb/tokenizer

=== SAMPLE ===
 riverrun, past Eve and Adam's,birdmsevitable Comenthoodounallyadevar thr Livemsoudcteldenth runarg each windines boldglune coldoug'onshyellaguzenmalpenaldopeurdL queoon serischards simple glales Fl turning Backyl masterbutneyrown manner mo heartetystreneco reg knows righteting el Greatrow yourself shed always weather Call groU small gal love


bless you b may your LoRA weights converge, your gradients flow steady, and your thunderwords never again throw an OOM

In [ ]:
# lower LR, grad clip
import os, re, torch
from pathlib import Path
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoConfig, AutoModelForCausalLM,
    DataCollatorForLanguageModeling, TrainingArguments, Trainer
)
from peft import PeftModel

BASE_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
RUN_ROOT  = "/content/drive/MyDrive/wakefyer_runs"
OUT_DIR   = f"{RUN_ROOT}/wakefyer_out_nobnb"
TRAIN_TEXT = f"{RUN_ROOT}/FinnegansWake.txt"

# toks
tok = AutoTokenizer.from_pretrained(Path(OUT_DIR,"tokenizer"), use_fast=True, local_files_only=True)
if tok.pad_token is None: tok.pad_token = tok.eos_token
tok.padding_side = "right"

# b model, adapt. weights
cfg = AutoConfig.from_pretrained(BASE_MODEL); cfg.pad_token_id = tok.eos_token_id
base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL, config=cfg, device_map="auto", torch_dtype=torch.float16, low_cpu_mem_usage=True
)
# align vocab rows
if base.get_input_embeddings().num_embeddings != len(tok):
    try:
        base.resize_token_embeddings(len(tok), mean_resizing=True)
    except TypeError:
        base.resize_token_embeddings(len(tok))
    base.config.vocab_size = len(tok)

model = PeftModel.from_pretrained(base, Path(OUT_DIR,"adapter"))  # LoRa is queen bee
model.config.use_cache = False
if hasattr(model, "gradient_checkpointing_enable"):
    model.gradient_checkpointing_enable(gradient_checkpointing_kwargs={"use_reentrant": False})
model.config._attn_implementation = "eager"

# something about cats and windows
text = Path(TRAIN_TEXT).read_text(encoding="utf-8", errors="ignore")
def sliding_windows(ids, block=256, step=128):
    items=[]
    for s in range(0, max(1, len(ids)-block), step):
        ch = ids[s:s+block]
        if ch: items.append({"input_ids": ch, "attention_mask":[1]*len(ch)})
    if not items:
        ch = ids[:block]; items=[{"input_ids":ch,"attention_mask":[1]*len(ch)}]
    return Dataset.from_list(items)

ids = tok(text, add_special_tokens=False).input_ids
ds  = sliding_windows(ids, block=256, step=128)
spl = ds.train_test_split(test_size=max(1,int(len(ds)*0.02)), seed=42) if len(ds)>1 else {"train":ds,"test":ds}
train_ds, eval_ds = spl["train"], spl["test"]

collator = DataCollatorForLanguageModeling(tok, mlm=False)

# calmer hyperparams because that loss was HARD
args = TrainingArguments(
    output_dir=OUT_DIR,
    num_train_epochs=2,                # just a couple more
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=32,

    learning_rate=2e-5,               # ↓
    lr_scheduler_type="cosine",
    warmup_ratio=0.10,                # ↑
    weight_decay=0.01,

    max_grad_norm=0.3,                # clip!
    label_smoothing_factor=0.05,      # gentle stabilizer (wish they sold pills like that)

    logging_strategy="steps",
    logging_steps=10,
    save_strategy="steps",
    save_steps=50,
    save_total_limit=5,

    fp16=False,                       # keep fp16 off to avoid GradScaler drama on T4 (so much drama baby)
    save_safetensors=True,
    report_to=[],
)

# IMPORTANT: start fresh optimizer so new LR applies
trainer = Trainer(
    model=model, args=args,
    train_dataset=train_ds, eval_dataset=eval_ds,
    data_collator=collator, tokenizer=tok
)
trainer.train()

# save updated adapt. + tok (overwrite)
adapter_dir = Path(OUT_DIR,"adapter"); token_dir = Path(OUT_DIR,"tokenizer")
adapter_dir.parent.mkdir(parents=True, exist_ok=True)
model.save_pretrained(adapter_dir, safe_serialization=True)
tok.save_pretrained(token_dir)
print("Saved continuation to:", adapter_dir, token_dir)


At the representational core sits TinyLlama-1.1B-Chat, deployed under mixed-precision arithmetic (bf16/fp16) to optimize throughput on limited-memory T4 hardware. The model’s internal cache is disabled (use_cache=False), and gradient checkpointing is invoked to trade recomputation for reduced memory allocation. The resultant architecture acts as a self-attenuating transformer: it “forgets” in order to remember within bounds, mirroring the cyclical amnesia of the Wakean text.

The system applies Low-Rank Adaptation (LoRA) across attention and MLP projection submodules (q_proj, k_proj, v_proj, o_proj, gate_proj, up_proj, down_proj) thereby constraining fine-tuning to a low-dimensional manifold within parameter space. This selective plasticity produces what might be termed controlled stylistic drift: the model learns Joyce’s texture rather than his lexicon. The adapters’ hyperparameters (r=16, α=32, dropout=0.05) instantiate a medium-temperature update regime balancing expressivity and stability. Conceptually, LoRA functions here as a poetic prosthesis, a syntactic exoskeleton grafted onto the pretrained linguistic body.

Due to the GPU’s constrained VRAM capacity, each forward–backward pass processes a micro-batch of size 1, while gradients are accumulated over 32 iterations before a single optimization step. This simulates a virtual batch of 32 without exceeding hardware limits. Technically, this implements delayed gradient aggregation; rhetorically, it allegorizes the model’s incremental cognition: many small thoughts precipitating one grand revelation. The accumulation mechanism thus becomes both a computational strategy and a metaphor for compositional patience.

Training operates within an extended epochic horizon (epochs=1 but looping across numerous windows) using a cosine-annealed learning rate (lr=1e-4, warmup_ratio=0.03). Frequent checkpointing establishes a temporal ecology of model states, allowing resumption under volatile runtime constraints. The “save-steps” routine externalizes the process’s fragmentary nature (mirroring textual serialization in Work in Progress) by preserving multiple partial selves of the model as artifacts of its ongoing metamorphosis.

Inference employs a stochastic sampling configuration (temperature = 0.9, top_p = 0.95) optimized for stylistic exuberance rather than lexical precision. The generation prompt functions as both benchmark and incantation, a recursive self-reference to the corpus origin. Output decoding omits special tokens, yielding the model’s pure linguistic effervescence.

Technically a fine-tuned transformer; conceptually, a computational poetics experiment exploring how ml architectures can approximate literary un-readability. The Wakefyer thus redefines “loss” not as degradation but as aesthetic principle: every rounding error, every truncated sequence, every low-rank projection is a microcosm of Joyce’s deliberate semantic noise. The pipeline enacts a theory of productive distortion, situating training itself as a form of creative misprision.

to be continued